<a href="https://colab.research.google.com/github/GaetanAm/Master-1-Machine-Learning/blob/main/notebooks/TD4bis/Exercice_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EXERCICE 2

## Load the dataset in your notebook and check if there are any missing values

In [4]:
# Importer les bibliothèques nécessaires
from google.colab import files
import pandas as pd

# Télécharger le fichier localement
uploaded = files.upload()


Saving creditcard.csv to creditcard.csv


HTTPError: HTTP Error 404: Not Found

In [5]:
# Charger le dataset à partir du fichier téléchargé
data = pd.read_csv(list(uploaded.keys())[0])

# Vérifier les premières lignes
print("Aperçu du dataset :")
print(data.head())

# Vérifier les valeurs manquantes
print("\nValeurs manquantes dans chaque colonne :")
print(data.isnull().sum())

Aperçu du dataset :
   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26 

No missing value, we have a proper dataset

## Create variables X and y, where X will contain the 30 features and y will be the target “Class”

In [6]:
# Définir X (caractéristiques) et y (cible)
X = data.iloc[:, :-1]  # Toutes les colonnes sauf la dernière (Class)
y = data['Class']      # Colonne cible

print(f"\nDimensions des données : X = {X.shape}, y = {y.shape}")



Dimensions des données : X = (284807, 30), y = (284807,)


## Split the data into a training set and a test set with a test set of 20%.
You need to import train_test_split from sklearn.model selection. By setting the stratify argument to data[”Class”], you’re instructing the
train_test_split function to preserve the class proportions in both the training and testing sets. This ensures
that both classes are represented in a balanced manner. You can check the number of values present in a set
with the method value_counts() to ensure that both classes are well represented in y_train and y_test.

In [7]:
from sklearn.model_selection import train_test_split

# Diviser les données en train et test avec stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Vérifier la répartition des classes
print("\nRépartition des classes dans y_train :")
print(y_train.value_counts(normalize=True))
print("\nRépartition des classes dans y_test :")
print(y_test.value_counts(normalize=True))



Répartition des classes dans y_train :
Class
0    0.998271
1    0.001729
Name: proportion, dtype: float64

Répartition des classes dans y_test :
Class
0    0.99828
1    0.00172
Name: proportion, dtype: float64


In [8]:
from sklearn.utils import resample

# Combiner X_train et y_train pour faciliter la manipulation
train_data = pd.concat([X_train, y_train], axis=1)

# Séparer les classes majoritaire et minoritaire
majority_class = train_data[train_data['Class'] == 0]
minority_class = train_data[train_data['Class'] == 1]

# Sous-échantillonnage de la classe majoritaire
majority_downsampled = resample(
    majority_class,
    replace=False,                # Échantillonnage sans remplacement
    n_samples=len(minority_class),  # Même nombre que la classe minoritaire
    random_state=42
)

# Combiner les deux classes
undersampled_data = pd.concat([majority_downsampled, minority_class])

# Répartition après sous-échantillonnage
print("\nRépartition des classes après sous-échantillonnage :")
print(undersampled_data['Class'].value_counts())

# Redéfinir X_train et y_train
X_train_undersampled = undersampled_data.iloc[:, :-1]
y_train_undersampled = undersampled_data['Class']



Répartition des classes après sous-échantillonnage :
Class
0    394
1    394
Name: count, dtype: int64


In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Modèle sur les données originales
model_original = LogisticRegression(max_iter=200, random_state=42)
model_original.fit(X_train, y_train)
y_pred_original = model_original.predict(X_test)

print("\nRapport de classification (données originales) :")
print(classification_report(y_test, y_pred_original))

# Modèle sur les données sous-échantillonnées
model_undersampled = LogisticRegression(max_iter=200, random_state=42)
model_undersampled.fit(X_train_undersampled, y_train_undersampled)
y_pred_undersampled = model_undersampled.predict(X_test)

print("\nRapport de classification (données sous-échantillonnées) :")
print(classification_report(y_test, y_pred_undersampled))


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c


Rapport de classification (données originales) :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.76      0.69      0.72        98

    accuracy                           1.00     56962
   macro avg       0.88      0.85      0.86     56962
weighted avg       1.00      1.00      1.00     56962


Rapport de classification (données sous-échantillonnées) :
              precision    recall  f1-score   support

           0       1.00      0.95      0.98     56864
           1       0.03      0.92      0.06        98

    accuracy                           0.95     56962
   macro avg       0.52      0.94      0.52     56962
weighted avg       1.00      0.95      0.97     56962

